# Simulation of a single 20-bead linear chain

In [2]:
import copy
import feasst as fst
print(fst.version())

def end_to_end_distance(mc):
    config = mc.system().configuration()
    pos = config.particle(0).site(0).position()
    return pos.distance(config.particle(0).site(config.particle(0).num_sites() - 1).position())

mc = fst.MonteCarlo()
mc.set(fst.MakeRandomMT19937(fst.args({"seed": "time"})))
mc.add(fst.MakeConfiguration(fst.args({"cubic_box_length": "30",
    "particle_type": fst.install_dir() + "/plugin/chain/forcefield/chain20.fstprt"})))
mc.add(fst.MakePotential(fst.MakeLennardJones(),
                         fst.MakeVisitModelIntra(fst.args({"cutoff": "1"}))))
mc.set(fst.MakeThermoParams(fst.args({"beta": "1", "chemical_potential0": "1"})))
mc.set(fst.MakeMetropolis())
mc.add(fst.MakeTrialAdd(fst.args({"particle_type": "0"})))
mc.run(fst.MakeRun(fst.args({"until_num_particles": "1"})))
mc.run(fst.MakeRemoveTrial(fst.args({"name": "TrialAdd"})))
mc.add(fst.MakeTrialPivot(fst.args({"weight": "1", "tunable_param": "20", "max_length": "8"})))
mc.add(fst.MakeTrialCrankshaft(fst.args({"weight": "1", "tunable_param": "20", "max_length": "8"})))
#mc.add(fst.MakeTrialReptate(fst.args({"weight": "1", "max_length": "1", "particle_type": "0"})))
unit_grows = list()
grows=list()
length=mc.configuration().num_sites()
max_length=19
for i in range(max_length):
     unit_grows.append([{"bond": "true", "mobile_site": str(i), "anchor_site": str(i+1), "num_steps": "4"}])
     if i == 0:
         grows.append(unit_grows[-1])
     else:
         grows.append(unit_grows[-1] + copy.deepcopy(grows[-1]))
unit_grows = list()
for i in range(max_length):
    unit_grows.append([{"bond": "true", "mobile_site": str(length-i-1), "anchor_site": str(length-i-2)}])
    if i == 0:
        grows.append(unit_grows[-1])
    else:
        grows.append(unit_grows[-1] + copy.deepcopy(grows[-1]))
for grow in grows:
    grow[0]["weight"] = str(1./len(grow))
    grow[0]["particle_type"] = "0"
    mc.add(fst.MakeTrialGrow(fst.ArgsVector(grow)))
steps_per = str(int(1e4))
mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "chain"})))
mc.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per": steps_per, "tolerance": str(1e-10)})))
print('Potential energy of initial configuration:', mc.criteria().current_energy())
rg_sq = fst.Accumulator()
print('end to end', end_to_end_distance(mc))
mc.attempt(int(1e3)) # equilibrate
for n in range(int(1e5)):
    mc.attempt()
    rg_sq.accumulate(end_to_end_distance(mc))

print(rg_sq.str())

v0.18.1-61-gca844632f9-hwh/txtin_factory
Potential energy of initial configuration: -2.022678100397566
end to end 12.767145334803702
average,stdev,block_stdev,moment0,moment1,moment2,moment3,moment4,
2.9101307491409747,1.0772850857103031,0.056074188907484161,100000,291013.07491409747,962939.25275581061,3533838.6102219024,14138621.456323513,


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!